In [1]:
from dotenv import load_dotenv
import os, requests, sys, json
from IPython.display import JSON
from pathlib import Path
import logging.handlers
from tqdm import tqdm
import pandas as pd
import numpy as np
from scipy.signal import convolve2d
import warnings
import seaborn as sns

from os import listdir
from os.path import isfile, join
warnings.filterwarnings("ignore")

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from Helpers.Exception_Handling import Exception_Handling as eh
from Helpers.Mongo_Interface import Mongo_Interface as mi

env_path = Path(fr"{os.environ['USERPROFILE']}\.env") 
load_dotenv(dotenv_path=env_path)

True

In [2]:
path = r'N:\connectxdocs'
onlyfiles = [f for f in tqdm(listdir(path)) if isfile(join(path, f))]

len(onlyfiles)

100%|█████████████████████████████████████████████████████████████████████████| 32586/32586 [00:00<00:00, 43851.52it/s]


32586

In [3]:
onlyfiles[0:5]

['45907088.json',
 '45907087.json',
 '45907086.json',
 '45907085.json',
 '45907084.json']

In [4]:
results = []
for file in tqdm(onlyfiles):
    try:
        document = json.load(open(f"{path}\\{file}"))
        if 'board' in document['steps'][-1:][0][0]['observation']:
            results.append(document)
    except:
        print(file)
len(results)

100%|██████████████████████████████████████████████████████████████████████████| 32586/32586 [00:11<00:00, 2869.49it/s]


32586

In [5]:
horizontal_kernel = np.array([[ 1, 1, 1, 1]])
vertical_kernel = np.transpose(horizontal_kernel)
diag1_kernel = np.eye(4, dtype=np.uint8)
diag2_kernel = np.fliplr(diag1_kernel)
detection_kernels = [horizontal_kernel, vertical_kernel, diag1_kernel, diag2_kernel]

def winning_move(board, player):
    for kernel in detection_kernels:
        if (convolve2d(board == player, kernel, mode="valid") == 4).any():
            return True
    return False

In [6]:
columns = [item for item in range(1, 43)]
columns.append('board_series')
columns.append('winner')
columns.append('looser')
columns.append('is_stale')


df = pd.DataFrame(columns=columns)
obj =''
for i in tqdm(range(len(results))):
    flat_board = results[i]['steps'][-1:][0][0]['observation']['board'].copy()
    if len(results[i]['steps']) < 3:
           continue
    board = np.resize(flat_board, (6,7))
    one_winning_move = winning_move(board, 1)
    flat_board.append(results[i]['steps'])
    if winning_move(board, 1):
        flat_board.append(1)
        flat_board.append(2)
        flat_board.append(False)
    elif winning_move(board, 2):
        flat_board.append(2)
        flat_board.append(1)
        flat_board.append(False)
    else:
        flat_board.append(None)
        flat_board.append(None)
        flat_board.append(True)
    df2 = pd.DataFrame(data=[flat_board], columns=columns)
    df = pd.concat([df, df2], ignore_index=True)
df

100%|███████████████████████████████████████████████████████████████████████████| 32586/32586 [01:52<00:00, 290.92it/s]


,1,2,3,4,5,6,7,8,9,10,...,37,38,39,40,41,42,board_series,winner,looser,is_stale
0,0,0,0,0,0,0,0,0,0,0,...,2,2,2,1,1,2,"[[{'action': 0, 'info': {}, 'observation': {'b...",1,2,False
1,0,0,0,0,0,0,0,0,0,2,...,1,2,1,2,2,1,"[[{'action': 0, 'info': {}, 'observation': {'b...",2,1,False
2,2,1,1,2,1,1,0,1,2,2,...,1,2,1,1,2,2,"[[{'action': 0, 'info': {}, 'observation': {'b...",None,None,True
3,0,0,0,0,0,0,0,2,1,0,...,1,2,1,1,2,1,"[[{'action': 0, 'info': {}, 'observation': {'b...",2,1,False
4,0,0,0,1,0,1,1,2,0,0,...,1,2,2,1,2,1,"[[{'action': 0, 'info': {}, 'observation': {'b...",1,2,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32515,0,0,0,0,0,2,0,0,0,2,...,1,1,2,2,2,1,"[[{'action': 0, 'info': {}, 'observation': {'b...",1,2,False
32516,0,0,0,0,0,0,0,0,0,0,...,2,2,2,2,1,1,"[[{'action': 0, 'info': {}, 'observation': {'b...",2,1,False
32517,0,2,0,0,0,0,0,0,1,0,...,2,1,1,2,2,1,"[[{'action': 0, 'info': {}, 'observation': {'b...",2,1,False
32518,0,0,0,0,0,0,0,2,0,0,...,1,2,1,2,2,0,"[[{'action': 0, 'info': {}, 'observation': {'b...",1,2,False


In [7]:
df[df['winner'] == 2]

,1,2,3,4,5,6,7,8,9,10,...,37,38,39,40,41,42,board_series,winner,looser,is_stale
1,0,0,0,0,0,0,0,0,0,2,...,1,2,1,2,2,1,"[[{'action': 0, 'info': {}, 'observation': {'b...",2,1,False
3,0,0,0,0,0,0,0,2,1,0,...,1,2,1,1,2,1,"[[{'action': 0, 'info': {}, 'observation': {'b...",2,1,False
8,2,0,0,0,0,0,1,1,0,0,...,1,2,2,1,2,1,"[[{'action': 0, 'info': {}, 'observation': {'b...",2,1,False
10,1,2,1,2,2,0,0,2,1,1,...,2,1,2,1,2,2,"[[{'action': 0, 'info': {}, 'observation': {'b...",2,1,False
11,1,0,1,1,0,1,0,1,0,1,...,2,2,1,2,2,0,"[[{'action': 0, 'info': {}, 'observation': {'b...",2,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32503,0,2,0,0,1,0,0,0,1,0,...,1,2,1,2,1,1,"[[{'action': 0, 'info': {}, 'observation': {'b...",2,1,False
32505,0,0,0,0,0,0,0,0,0,0,...,2,0,2,0,0,1,"[[{'action': 0, 'info': {}, 'observation': {'b...",2,1,False
32510,0,0,0,0,0,0,0,0,0,0,...,1,0,1,1,0,2,"[[{'action': 0, 'info': {}, 'observation': {'b...",2,1,False
32516,0,0,0,0,0,0,0,0,0,0,...,2,2,2,2,1,1,"[[{'action': 0, 'info': {}, 'observation': {'b...",2,1,False


In [8]:
#df.loc[2, 'board_series']

In [9]:
columns2 = [item for item in range(1, 43)]
columns2.append('target')
columns2.append('winner')

df_actions = pd.DataFrame(columns=columns2)
working_set = df[df['is_stale'] == False]

preload = []

for i in tqdm(working_set.index):
    series = working_set.loc[i, 'board_series'].copy()
    winning_player = working_set.loc[i, 'winner']
    if len(series) > 3:
        if winning_player == 1:
            for j in range(2, len(series), 2):
                if len(series) > j + 1:
                    flat_board = working_set.loc[i, 'board_series'][j][0]['observation']['board'].copy()
                    #print(len(flat_board))
                    seen = np.resize(working_set.loc[i, 'board_series'][j][0]['observation']['board'], (6,7))
                    action = np.resize(working_set.loc[i, 'board_series'][j + 1][0]['observation']['board'], (6,7))
                    result = action - seen
                    target = np.unravel_index(result.argmax(), result.shape)[1]
                    flat_board.append(target)
                    flat_board.append(winning_player)
                    #print(flat_board)
                    preload.append(flat_board)
                    #preload.append(winning_player)
                    #df2 = pd.DataFrame(data=[flat_board], columns=columns2)
        elif winning_player == 2:
            for j in range(1, len(series), 2):
                if len(series) > j + 1:
                    flat_board = working_set.loc[i, 'board_series'][j][0]['observation']['board'].copy()
                    #print(len(flat_board))
                    seen = np.resize(working_set.loc[i, 'board_series'][j][0]['observation']['board'], (6,7))
                    action = np.resize(working_set.loc[i, 'board_series'][j + 1][0]['observation']['board'], (6,7))
                    result = action - seen
                    target = np.unravel_index(result.argmax(), result.shape)[1]
                    flat_board.append(target)
                    flat_board.append(winning_player)
                    #print(flat_board)
                    preload.append(flat_board)
                    #preload.append(winning_player)
                    #df2 = pd.DataFrame(data=[flat_board], columns=columns2)

df_actions = pd.DataFrame(data=preload, columns=columns2)

df_actions

100%|██████████████████████████████████████████████████████████████████████████| 30721/30721 [00:10<00:00, 3046.76it/s]


,1,2,3,4,5,6,7,8,9,10,...,35,36,37,38,39,40,41,42,target,winner
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,2,0,0,0,0,4,1
1,0,0,0,0,0,0,0,0,0,0,...,0,1,2,2,0,1,0,0,2,1
2,0,0,0,0,0,0,0,0,0,0,...,0,1,2,2,0,1,0,2,5,1
3,0,0,0,0,0,0,0,0,0,0,...,0,1,2,2,2,1,1,2,5,1
4,0,0,0,0,0,0,0,0,0,0,...,2,1,2,2,2,1,1,2,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358161,0,0,0,0,0,0,0,0,0,0,...,0,2,1,2,1,2,2,0,5,1
358162,0,0,0,0,0,0,0,0,0,0,...,0,2,1,2,1,2,2,0,2,1
358163,0,0,0,0,0,0,0,0,0,0,...,0,2,1,2,1,2,2,0,6,1
358164,0,0,0,2,0,0,0,0,0,0,...,0,2,1,2,1,2,2,1,4,1


In [10]:
df_actions['winner'].unique()

array([1, 2], dtype=int64)

In [11]:
df_actions['tick'] = 1

In [12]:
columns2 = [item for item in range(1, 43)]
columns2.append('target')
columns2.append('winner')
df_actions_set_one = df_actions.groupby(columns2)['tick'].sum().sort_values(ascending=False).reset_index()
df_actions_set_one['tick_layer_two'] = 1
df_actions_set_one
#df_actions_set_one.groupby(columns2)['tick_layer_two'].sum().sort_values(ascending=False)

,1,2,3,4,5,6,7,8,9,10,...,37,38,39,40,41,42,target,winner,tick,tick_layer_two
0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,3,1,2508,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,3,2,1590,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,3,1,1186,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,3,2,638,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,3,2,536,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227770,0,0,0,0,0,0,0,0,0,0,...,1,2,0,1,0,0,0,1,1,1
227771,0,0,0,0,0,0,0,0,0,0,...,1,2,1,0,1,2,2,1,1,1
227772,0,0,0,0,0,0,0,0,0,0,...,1,2,2,1,1,1,4,1,1,1
227773,0,0,0,0,0,0,0,0,0,0,...,1,2,2,1,1,1,6,1,1,1


In [13]:
df_actions_set_two = df_actions_set_one.reset_index()
df_actions_set_two = df_actions_set_two.drop(columns=['index'])

columns3 = [item for item in range(1, 43)]
columns3.append('winner')

df_actions_set_two = df_actions_set_two.drop_duplicates(subset=columns3, keep="first").reset_index()
df_actions_set_two = df_actions_set_two.drop(columns=['index','tick', 'tick_layer_two'])
df_actions_set_two

,1,2,3,4,5,6,7,8,9,10,...,35,36,37,38,39,40,41,42,target,winner
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,3,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,3,2
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,3,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,3,2
4,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210406,0,0,0,0,0,0,0,0,0,0,...,0,1,1,2,0,1,0,0,0,1
210407,0,0,0,0,0,0,0,0,0,0,...,0,1,1,2,1,0,1,2,2,1
210408,0,0,0,0,0,0,0,0,0,0,...,1,2,1,2,2,1,1,1,4,1
210409,0,0,0,0,0,0,0,0,0,0,...,1,2,1,2,2,1,1,1,6,1


In [14]:
df_actions_set_two.to_csv('N:\\connectxactions\\actions.csv')

In [15]:
df_actions_set_two.to_json('N:\\connectxactions\\actions.json')

In [23]:
sub_template = ''
with open('N:\\connectxactions\\bot_template.txt') as f:
    sub_template = f.read()

cavdata = []
with open('N:\\connectxactions\\actions.csv') as f:
    cavdata = f.read()



In [24]:
from io import StringIO

In [25]:
TESTDATA = StringIO("""col1;col2;col3
    1;4.4;99
    2;4.5;200
    3;4.7;65
    4;3.2;140
    """)

pd.read_csv(TESTDATA, sep=";")

,col1,col2,col3
0,1,4.4,99
1,2,4.5,200
2,3,4.7,65
3,4,3.2,140


In [26]:
TESTDATA = StringIO(cavdata)
columns4 = [item for item in range(1, 43)]
columns4.append('winner')

pd.read_csv(TESTDATA, index_col = columns4)

Unnamed: 0  \
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 winner               
0 0 0 0 0 0 0 0 0 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  2  0  0  0  0  0  0  1  0  0  0  1                0   
                                                                                    0  0  0  0  0  0  0  1  0  0  0  2                1   
                                          2  0  0  0  0  0  0  1  0  0  0  0  0  0  2  0  0  0  0  0  0  1  0  0  0  1                2   
                                          0  0  0  0  0  0  0  1  0  0  0  0  0  0  2  0  0  0  0  0  0  1  0  0  0  2                3   
                                                               0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  2                4   
...                                                                                                                                 ...   
                                       1  0  0  0  0  0  2  2  0  0  0  0  2  1  2  0  0  0  0  1  1  2  0  1  0  0  1           210406   
                                                                                                         1  0  1  2  1           210407   
                                                                              2  1  0  0  0  1  2  1  2  2  1  1  1  1           210408   
                                                                                       1  2  1  2  1  2  2  1  1  1  1           210409   
2 2 2 1 2 1 0 1 1 1  2  2  2  0  2  2  1  2  1  1  0  1  1  2  1  1  2  0  2  1  1  2  2  2  0  1  2  1  1  1  2  1  2           210410   

                                                                                                                             target  
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 winner          
0 0 0 0 0 0 0 0 0 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  2  0  0  0  0  0  0  1  0  0  0  1            3  
                                                                                    0  0  0  0  0  0  0  1  0  0  0  2            3  
                                          2  0  0  0  0  0  0  1  0  0  0  0  0  0  2  0  0  0  0  0  0  1  0  0  0  1            3  
                                          0  0  0  0  0  0  0  1  0  0  0  0  0  0  2  0  0  0  0  0  0  1  0  0  0  2            3  
                                                               0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  2            3  
...                                                                                                                             ...  
                                       1  0  0  0  0  0  2  2  0  0  0  0  2  1  2  0  0  0  0  1  1  2  0  1  0  0  1            0  
                                                                                                         1  0  1  2  1            2  
                                                                              2  1  0  0  0  1  2  1  2  2  1  1  1  1            4  
                                                                                       1  2  1  2  1  2  2  1  1  1  1            6  
2 2 2 1 2 1 0 1 1 1  2  2  2  0  2  2  1  2  1  1  0  1  1  2  1  1  2  0  2  1  1  2  2  2  0  1  2  1  1  1  2  1  2            6  

[210411 rows x 2 columns]

In [27]:
sub_file = sub_template.replace('<REPLACEMEWITHDATA>',f"""
TESTDATA = StringIO('''{cavdata}''')
columns4 = [item for item in range(1, 43)]
columns4.append('winner')
df_map = pd.read_csv(TESTDATA, index_col = columns4)
""")


In [28]:
with open('N:\\connectxactions\\sub.py', 'w') as f:
    f.write(sub_file)

In [29]:
sub_template

'from kaggle_environments import evaluate, make, utils\nimport pandas as pd\nimport numpy as np\nimport random\nimport requests\nfrom tqdm import tqdm\nfrom scipy.signal import convolve2d\nimport sys\nfrom io import StringIO\n\n#result = requests.get(\'https://www.kaggleusercontent.com/episodes/45908179.json\')\n#sys.stdout.write(\'googleresult\')\n#sys.stdout.write(str(len(result.text)))\n\n<REPLACEMEWITHDATA>\n\n\nhorizontal_kernel = np.array([[ 1, 1, 1, 1]])\nvertical_kernel = np.transpose(horizontal_kernel)\ndiag1_kernel = np.eye(4, dtype=np.uint8)\ndiag2_kernel = np.fliplr(diag1_kernel)\ndetection_kernels = [horizontal_kernel, vertical_kernel, diag1_kernel, diag2_kernel]\n\ndef winning_move(board, player):\n    for kernel in detection_kernels:\n        if (convolve2d(board == player, kernel, mode="valid") == 4).any():\n            return True\n    return False\n\ndef smart_move(board, me, enemey, recurse_count = 0, target_chosen=0):\n    sys.stdout.write(f\'me:{me} enemy:{enemey}\